# BarrelNet Inference Notebook
This Notebook will run the code to run an inference pipeline for the trained Model, given a single Barrel point cloud sample as input. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.optim as optim
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import trimesh
import plotly.graph_objects as go
import uuid
import roma
import visu3d as v3d
import scipy.linalg
from barrelnet.pointnet.data import generate_cylinder_pts, prepare_point_cloud, normalize_pc, CylinderData
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
from barrelnet.pointnet.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet.pointnet.barrelnet import BarrelNet
from barrelnet.pointnet.data import pad_point_cloud
from barrelnet.synthbarrel import random_cylinder_vol, random_cylinder_surf, monte_carlo_volume_ratio, generate_oriented_barrel, get_cyl_endpoints, get_cylinder_surf

In [ ]:
def inference_format(points, max_points=1000):
    """ Convert pts in the right format for input to pointnet
    Args:
        points: [N,3] torch tensor

    Returns:
        pts [1,N,3]
        scale 
    """
    pts, scale = normalize_pc(points)
    pts = pad_point_cloud(pts, max_points)
    pts = pts.permute(1,0)
    pts = pts.unsqueeze(0)
    return pts, scale

In [ ]:
## Load Model 
model_path = "checkpoints/pointnet_iter7000.pth"
height_ratio = 2.5  # height / radius ratio
pointnet = BarrelNet(k=5, normal_channel=False).cuda()
pointnet.load_state_dict(torch.load(model_path))
pointnet.eval()

In [ ]:
# cylnp = random_cylinder_surf([0, 0, 0], [0, 0, height_ratio], 1, 5000).astype(np.float32)
cylh = 2.5
cylr = cylh / height_ratio
ntrials = 5000

axtruths = []
axpreds = []
ztruths = []
zpreds = []
rtruths = []
rpreds = []
pcs = []
cos_sims = []
burialtruths = []
burialpreds = []
burial_errs = []
trialresults = []
for i in tqdm(range(ntrials)):
    results = {}
    cylnp, _, cylax, cylz = generate_oriented_barrel(cylr, cylh, 5000, sigma=0.05, zlims=[-0.3, 0.3])
    x1truth, x2truth = get_cyl_endpoints(cylax, height_ratio, cylz)
    burialtruth = monte_carlo_volume_ratio(10000, x1truth, x2truth, cylr, 0, 0, 1, 0)
    results["axtruth"] = cylax
    results["pc"] = cylnp
    results["ztruth"] = cylz
    results["rtruth"] = cylr
    results["x1truth"] = x1truth
    results["x2truth"] = x2truth
    results["burialtruth"] = burialtruth

    cylnp = cylnp.astype(np.float32)
    pts = torch.from_numpy(cylnp).cuda()
    pts, scale = inference_format(pts)
    with torch.no_grad():
        radius_pred, zshift_pred, axis_pred = pointnet(pts)
        radius_pred = radius_pred.cpu().numpy()[0]
        zshift_pred = zshift_pred.cpu().numpy()[0]
        axis_pred = axis_pred.cpu().numpy()[0]
    axis_pred = axis_pred / np.linalg.norm(axis_pred)
    axpreds.append(axis_pred)
    r = radius_pred * scale
    h = r * height_ratio
    z = zshift_pred * h
    zpreds.append(z)
    rpreds.append(r)
    x1, x2 = get_cyl_endpoints(axis_pred, h, z)
    burialpred = monte_carlo_volume_ratio(10000, x1, x2, r, 0, 0, 1, 0)
    burialpreds.append(burialpred)
    
    cos_sims.append(np.abs(cylax @ axis_pred))
    burial_errs.append(np.abs(burialtruth - burialpred))

    # print("TRUTH")
    # print(f"axis: {cylax}\nradius: {cylr}\nheight: {cylh}\nz-offset: {cylz}")
    # print(f"burial percentage: {burialtruth}")
    # print("PREDICTED")
    # print(radius_pred, zshift_pred, axis_pred)
    # print(f"axis: {axis_pred}\nradius: {r}\nheight: {h}\nz-offset: {z}")
    # print(f"burial percentage: {burialpred}")

    # truthray = v3d.Ray(pos=[0,0,0], dir=cylax)
    # predray = v3d.Ray(pos=[0,0,0], dir=axis_pred)
    # v3d.make_fig([v3d.Point3d(p=cylnp), truthray, predray])

In [ ]:
cylz

In [ ]:
print(np.mean(cos_sims), np.std(cos_sims))
print(np.mean(burial_errs), np.std(burial_errs))

In [ ]:
np.sum(np.array(cos_sims) < 0.9)

In [ ]:
plt.hist(burialtruths)

In [ ]:
plt.hist(burial_errs)

In [ ]:
zerrs = np.abs(np.array(ztruths) - zpreds)
plt.hist(zerrs)

In [ ]:
rerrs = np.abs(np.array(rtruths) - rpreds)
plt.hist(rerrs)

In [ ]:
worstidx = np.argmin(cos_sims)
worstidx = 8
truthray = v3d.Ray(pos=[0,0,0], dir=axtruths[worstidx])
predray = v3d.Ray(pos=[0,0,0], dir=axpreds[worstidx])
v3d.make_fig([v3d.Point3d(p=pcs[worstidx]), truthray, predray])

In [ ]:
x, y, z = get_cylinder_surf([0, 0, 0], [1, 2, 1], 1)

cyl1 = go.Surface(
    x=x, y=y, z=z,
    #  colorscale = colorscale,
    #  showscale=False,
    opacity=0.5)
fig = go.Figure(data=[cyl1])
fig.show()